In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\Users\\Study\\anaconda3\\lib\\site-packages\\pyspark'

In [2]:
import os
cwd = os.getcwd()
print(cwd)


C:\Users\Study\Documents\Jupyter Notebooks\PoliTO\1-30 MapReduce Exercises\Exercise-4


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Exercise 4") \
    .getOrCreate()

In [4]:
spark

In [5]:
sc = spark.sparkContext

In [6]:
sc

<SparkContext master=local[*] appName=Exercise 4>

In [7]:
#PM10 pollution analysis per city zone
#* Input: a (structured) textual file containing the daily value of PM10 for a set of city zones
#* Each line of the file has the following format
#zoneId,date\tPM10 value (μg/m3 )\n
#* Output: report for each zone the list of dates associated with a PM10 value above a specific threshold
#* Suppose to set threshold = 50 μg/m3

In [8]:
rdd4_raw = sc.textFile("input_file_4.txt")

In [9]:
rdd4_raw.take(10)

['zone1,2016-01-01\t20.5',
 'zone2,2016-01-01\t30.1',
 'zone1,2016-01-02\t60.2',
 'zone2,2016-01-02\t20.4',
 'zone1,2016-01-03\t55.5',
 'zone2,2016-01-03\t52.5']

In [10]:
#eventually, refactor into a function...
rdd4_processing = rdd4_raw.map(lambda x: x.replace('\t',','))\
                          .map(lambda x: x.split(','))\
                          .map(lambda x: [x[0],x[1],float(x[2])])

rdd4_processing.take(10)


[['zone1', '2016-01-01', 20.5],
 ['zone2', '2016-01-01', 30.1],
 ['zone1', '2016-01-02', 60.2],
 ['zone2', '2016-01-02', 20.4],
 ['zone1', '2016-01-03', 55.5],
 ['zone2', '2016-01-03', 52.5]]

In [11]:
#rdd4 = rdd4.map(lambda x: x[0].split(','))
#rdd4.take(4)

In [16]:
rdd4_processing

PythonRDD[6] at RDD at PythonRDD.scala:53

In [19]:
rdd4_processing = rdd4_processing.filter(lambda x: x[2] > 50)

In [20]:
rdd4_processing.take(10)

[['zone1', '2016-01-02', 60.2],
 ['zone1', '2016-01-03', 55.5],
 ['zone2', '2016-01-03', 52.5]]

In [41]:
rdd4_op1 = rdd4_processing.map(lambda x: (x[0], x[1]))
rdd4_op1.take(10)
#rdd4_op1 = rdd4_op1.groupByKey()
#rdd4_processing_result = rdd4_processing.groupByKey().mapValues(list)

[('zone1', '2016-01-02'), ('zone1', '2016-01-03'), ('zone2', '2016-01-03')]

In [42]:
#combineByKey(lambda v:[v],lambda x,y:x+[y],lambda x,y:x+y).collect()


def to_list(a):
    return [a]

def append(a, b):
    a.append(b)
    return a

def extend(a, b):
    a.extend(b)
    return a

sorted(rdd4_op1.combineByKey(to_list, append, extend).collect())
#rdd4_op1 = rdd4_op1.combineByKey(to_list, append, extend)

[('zone1', ['2016-01-02', '2016-01-03']), ('zone2', ['2016-01-03'])]

In [56]:
rdd4_processing.take(10)
rdd4_op2 = rdd4_processing.map(lambda x:(x[0],x[1]))

In [57]:
rdd4_op2.take(10)

[('zone1', '2016-01-02'), ('zone1', '2016-01-03'), ('zone2', '2016-01-03')]

In [64]:
rdd4_op2_result = rdd4_op2.reduceByKey(lambda x,y: [x]+[y]).collect()

In [65]:
rdd4_op2_result

[('zone2', '2016-01-03'), ('zone1', ['2016-01-02', '2016-01-03'])]